# CME538 - Introduction to Data Science
## Lecture 6.2 - Gradient Descent I

## Setup Notebook

In [1]:
# Import 3rd party libraries
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import cm
import matplotlib.pylab as plt
from ipywidgets import interact, fixed, IntSlider

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

# Import Tips Data
Let's import out tips dataset from `Seaborn`.

In [ ]:
tips = sns.load_dataset('tips')
tips = tips.iloc[0:-1]
tips.head()

Add a column representing the tip percentage.

In [ ]:
tips['tip_pct'] = tips['tip'] / tips['total_bill'] * 100
tips.head()

In [ ]:
ax = sns.distplot(tips['tip_pct'])
ax.set_xlabel('Tip Percentage')

# Gradient Descent

In [5]:
def mse(theta, y_vals):
    return np.mean((y_vals - theta) ** 2)

def grad_mse(theta, y_vals):
    return -2 * np.mean(y_vals - theta)

In [6]:
def minimize(loss_fn, grad_loss_fn, dataset, alpha=0.2, progress=True):
    """
    Uses gradient descent to minimize loss_fn. 
    Returns the minimizing value of theta once 
    theta changes less than 0.001 between iterations.
    """
    # Set starting theta
    theta = 0
    
    # Set starting epoch
    epoch = 0
    
    while True:
        
        # Calculate the loss
        loss = loss_fn(theta, dataset)
        
        if progress:
            print('epoch: {} | theta: {} | loss: {}'.format(epoch, 
                                                            theta, 
                                                            loss))
        
        # Compute gradient
        gradient = grad_loss_fn(theta, dataset)
        
        # Get new theta
        new_theta = theta - alpha * gradient
        
        # Check for convergence
        if abs(new_theta - theta) < 0.001:
            return new_theta
        
        # Update theta
        theta = new_theta
        
        # Update epoch
        epoch += 1
        
    else:
        return theta 

In [ ]:
%%time
theta = minimize(loss_fn=mse, 
                 grad_loss_fn=grad_mse, 
                 dataset=tips['tip_pct'].values, 
                 alpha=0.2, progress=True)

print('\nMinimizing theta: {}'.format(theta))
print()

In [ ]:
%%time
theta = minimize(loss_fn=mse, 
                 grad_loss_fn=grad_mse, 
                 dataset=tips['tip_pct'].values, 
                 alpha=0.9, progress=True)

print('\nMinimizing theta: {}'.format(theta))
print()

In [9]:
def minimize_plot_constant_model(epochs, loss_fn, grad_loss_fn, dataset, alpha=0.2):
    """
    Uses gradient descent to minimize loss_fn. Returns the minimizing value of
    theta once theta changes less than 0.001 between iterations.
    """
    # Set starting theta
    theta = 90
    
    # Set starting epoch
    epochs = np.arange(epochs)
    epochs_conv = []
    
    # Set loss array
    losses = []
    losses_conv = []
    
    # Set theta array
    thetas = []
    
    # Setup figure
    fig = plt.figure(figsize=(12, 5))
    fig.subplots_adjust(wspace=0.2, hspace=0.2)
    ax1 = plt.subplot2grid((1, 2), (0, 0))
    ax2 = plt.subplot2grid((1, 2), (0, 1))
    
    # Plot loss 
    sns.rugplot(tips['tip_pct'].values, height=0.1, ax=ax1)
    ax1.plot(np.arange(-100, 120, 0.01), [loss_fn(val, dataset) for val in np.arange(-100, 120, 0.01)], '-')
    ax1.set_xlabel('$\\theta$', fontsize=18)
    ax1.set_ylabel('Loss', fontsize=18)
    ax1.set_xlim([-60, 102])
    ax1.set_ylim([-1000, 8000])
    ax1.xaxis.set_tick_params(labelsize=14)
    ax1.yaxis.set_tick_params(labelsize=14)

    for epoch in epochs:
        
        # Update losses
        losses.append(loss_fn(theta, dataset))
        
        # Update thetas
        thetas.append(theta)
        
        # Compute gradient
        gradient = grad_loss_fn(theta, dataset)
        
        # Get new theta
        new_theta = theta - alpha * gradient
        
        if abs(new_theta - theta) < 0.001:
            epochs_conv.append(epoch)
            losses_conv.append(loss_fn(theta, dataset))
          
        # Update theta
        theta = new_theta
  
    # Check for convergence
    ax2.plot(epochs, losses, '-o', markeredgecolor='k', ms=10)
    ax2.plot(epochs_conv, losses_conv, '-o', markeredgecolor='k', ms=10)
    ax2.set_xlim([-0.5, 50])
    ax2.set_ylim([-200, 8000])
    ax2.set_xlabel('Epoch', fontsize=18)
    ax2.set_ylabel('Loss', fontsize=18)
    ax2.xaxis.set_tick_params(labelsize=14)
    ax2.yaxis.set_tick_params(labelsize=14)

    ax1.set_title('$\\theta$ = {}'.format(np.round(theta, 2), fontsize=18))
    ax1.plot(thetas, losses, '-o', markeredgecolor='k', ms=10)
    
    xs = np.arange(thetas[-1] - 30, thetas[-1] + 30, 0.05)
    ys = losses[-1] + gradient * (xs - thetas[-1])
    ax1.plot(xs, ys, zorder=0, linestyle='--', lw=2)
    
    plt.show()

In [ ]:
alpha = 1.01

# Launch widget
interact(minimize_plot_constant_model,
         epochs=IntSlider(value=1, min=1, max=100, step=1),
         loss_fn=fixed(mse),
         grad_loss_fn=fixed(grad_mse),
         dataset=fixed(tips['tip_pct'].values),
         alpha=fixed(alpha));

# 1D Gradient  Descent

In [ ]:
plt.figure(figsize=(10, 5))
ax = sns.scatterplot(x = tips['total_bill'], y = tips['tip'])
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
ax.set_xlabel('Total Bill', fontsize=18)
ax.set_ylabel('Tip', fontsize=18)
plt.show()

In [14]:
def mse(theta, x, y_obs):
    y_hat = theta * x
    return np.mean((y_hat - y_obs) ** 2)

def grad_mse(theta, x, y_obs):
    y_hat = theta * x
    return np.mean(2 * (y_hat - y_obs) * x)

In [17]:
def minimize_plot_1d_model(epochs, loss_fn, grad_loss_fn, x, y_obs, alpha=0.2):
    """
    Uses gradient descent to minimize loss_fn. Returns the minimizing value of
    theta once theta changes less than 0.001 between iterations.
    """
    # Set starting theta
    theta = 90
    
    # Set starting epoch
    epochs = np.arange(epochs)
    epochs_conv = []
    
    # Set loss array
    losses = []
    losses_conv = []
    
    # Set theta array
    thetas = []
    
    # Setup figure
    fig = plt.figure(figsize=(14, 12))
    fig.subplots_adjust(wspace=0.2, hspace=0.2)
    ax1 = plt.subplot2grid((2, 2), (0, 0), colspan=2)
    ax2 = plt.subplot2grid((2, 2), (1, 0))
    ax3 = plt.subplot2grid((2, 2), (1, 1))
    
    # Plot total_bill vs tip
    sns.scatterplot(x = x, y = y_obs, ax=ax1)
    ax1.xaxis.set_tick_params(labelsize=14)
    ax1.yaxis.set_tick_params(labelsize=14)
    ax1.set_xlabel('Total Bill', fontsize=18)
    ax1.set_ylabel('Tip', fontsize=18)
    ax1.set_xlim([0, 60])
    ax1.set_ylim([0, 12])
    
    # Plot loss 
    ax2.plot(np.arange(-100, 120, 0.01), [loss_fn(val, x, y_obs) for val in np.arange(-100, 120, 0.01)], '-')
    ax2.set_xlabel('$\\theta$', fontsize=18)
    ax2.set_ylabel('Loss', fontsize=18)
    #ax2.set_xlim([-60, 102])
    #ax2.set_ylim([-1000, 8000])
    ax2.xaxis.set_tick_params(labelsize=14)
    ax2.yaxis.set_tick_params(labelsize=14)

    for epoch in epochs:
        
        # Update losses
        losses.append(loss_fn(theta, x, y_obs))
        
        # Update thetas
        thetas.append(theta)
        
        # Compute gradient
        gradient = grad_loss_fn(theta, x, y_obs)
        
        # Get new theta
        new_theta = theta - alpha * gradient
        
        if abs(new_theta - theta) < 0.001:
            epochs_conv.append(epoch)
            losses_conv.append(loss_fn(theta, x, y_obs))
          
        # Update theta
        theta = new_theta
  
    # Check for convergence
    ax3.plot(epochs, losses, '-o', markeredgecolor='k', ms=10)
    ax3.plot(epochs_conv, losses_conv, '-o', markeredgecolor='k', ms=10)
    ax3.set_xlim([-0.5, 20])
    #ax3.set_ylim([-200, 8000])
    ax3.set_xlabel('Epoch', fontsize=18)
    ax3.set_ylabel('Loss', fontsize=18)
    ax3.xaxis.set_tick_params(labelsize=14)
    ax3.yaxis.set_tick_params(labelsize=14)

    ax2.plot(thetas, losses, '-o', markeredgecolor='k', ms=10)
    
    xs = np.arange(thetas[-1] - 30, thetas[-1] + 30, 0.05)
    ys = losses[-1] + gradient * (xs - thetas[-1])
    ax2.plot(xs, ys, zorder=0, linestyle='--', lw=2)
    
    ax1.plot(x, x*theta, linestyle='-', lw=3, color='#fc4f30')
    if len(losses_conv) > 0:
        ax1.vlines(x, ymin=y_obs, ymax=x*theta, linestyle='dashed', color='r',alpha=0.3, zorder=0)

    plt.show()

In [ ]:
alpha = 0.0005

# Launch widget
interact(minimize_plot_1d_model,
         epochs=IntSlider(value=1, min=1, max=20, step=1),
         loss_fn=fixed(mse),
         grad_loss_fn=fixed(grad_mse),
         x=fixed(tips['total_bill'].values),
         y_obs=fixed(tips['tip'].values),
         alpha=fixed(alpha));

# 2D Gradient  Descent (Unscaled)

In [19]:
def mse(theta0, theta1, x, y_obs):
    y_hat = theta0 + theta1 * x
    return np.mean((y_hat - y_obs) ** 2)

def grad_mse(theta0, theta1, x, y_obs):
    y_hat = theta0 + theta1 * x
    n = len(x)
    grad_0 = (-2 / n) * sum(y_obs - y_hat)
    grad_1 = (-2 / n) * sum(x * (y_obs - y_hat)) 
    return grad_0, grad_1

In [22]:
def minimize_plot_2d_model_unscaled(epochs, loss_fn, grad_loss_fn, x, y_obs, alpha=0.2):
    """
    Uses gradient descent to minimize loss_fn. Returns the minimizing value of
    theta once theta changes less than 0.001 between iterations.
    """
    
    # Set starting epoch
    epochs = np.arange(epochs)
    epochs_conv = []
    
    # Set loss array
    losses = []
    losses_conv = []
    
    # Set theta array
    theta0s = []
    theta1s = []
    
    # Setup figure
    fig = plt.figure(figsize=(14, 12))
    fig.subplots_adjust(wspace=0.2, hspace=0.2)
    ax1 = plt.subplot2grid((2, 2), (0, 0))
    ax2 = plt.subplot2grid((2, 2), (0, 1))
    ax3 = plt.subplot2grid((2, 2), (1, 0))
    ax4 = plt.subplot2grid((2, 2), (1, 1), projection='3d')
    
    # Plot total_bill vs tip
    sns.scatterplot(x = x, y = y_obs, ax=ax1)
    ax1.xaxis.set_tick_params(labelsize=14)
    ax1.yaxis.set_tick_params(labelsize=14)
    
    ax1.set_ylabel('Tip', fontsize=18)
    ax1.set_xlabel('Total Bill', fontsize=18)
    ax1.set_xlim([0, 60])
    ax1.set_ylim([0, 12])
    
    # Plot 2D contour
    theta0_grid = np.linspace(-5, 15, 200)
    theta1_grid = np.linspace(-10, 11, 200)
    loss_grid = np.zeros([len(theta1_grid), len(theta0_grid)])
    for theta0_id, theta0 in enumerate(theta0_grid):
        for theta1_id, theta1 in enumerate(theta1_grid):
            loss_grid[theta1_id, theta0_id] = loss_fn(theta0, theta1, x, y_obs)
    X, Y = np.meshgrid(theta0_grid, theta1_grid)
    cp = ax3.contour(X, Y, loss_grid, 6, colors='white', linestyles='dashed', linewidths=1)
    ax3.clabel(cp, inline=1, fmt='%1.1f', fontsize=15 )
    ax3.contourf(X, Y, loss_grid, alpha=0.65,cmap=cm.viridis) 
    ax3.set_xlabel('$\\theta_0$', fontsize=18)
    ax3.set_ylabel('$\\theta_1$', fontsize=18)

    surf = ax4.plot_surface(X, Y, loss_grid, cmap='viridis', rstride=1, cstride=1, antialiased=True, alpha=0.65)
    ax4.set_xlabel('$\\theta_0$', fontsize=18)
    ax4.set_ylabel('$\\theta_1$', fontsize=18)
    ax4.set_zlabel('Loss', fontsize=18, rotation=90)
    
    # Set starting theta
    theta0 = 10.
    theta1 = 10.
    
    for epoch in epochs:

        # Update losses
        losses.append(loss_fn(theta0, theta1, x, y_obs))
        
        # Update thetas
        theta0s.append(theta0)
        theta1s.append(theta1)
        
        # Compute gradient
        gradient_0, gradient_1 = grad_loss_fn(theta0, theta1, x, y_obs)
        
        # Get new theta
        new_theta0 = theta0 - alpha * gradient_0
        new_theta1 = theta1 - alpha * gradient_1
        
        if abs(new_theta0 - theta0) < 0.001 and abs(new_theta1 - theta1) < 0.001:
            epochs_conv.append(epoch)
            losses_conv.append(loss_fn(theta0, theta1, x, y_obs))
          
        # Update theta
        theta0 = new_theta0
        theta1 = new_theta1
  
    # Check for convergence
    ax2.plot(epochs, losses, '-o', markeredgecolor='k', ms=10)
    ax2.set_xlim([-1, 10000])
    ax2.set_ylim([-20, 400])
    ax2.set_xlabel('Epoch', fontsize=18)
    ax2.set_ylabel('Loss', fontsize=18)
    ax2.xaxis.set_tick_params(labelsize=14)
    ax2.yaxis.set_tick_params(labelsize=14)
    
    ax1.set_title('$\\theta_0$ = {}\n$\\theta_1$ = {}'.format(np.round(theta0, 2), np.round(theta1, 2)), fontsize=18)
    ax1.plot(x, theta0 + x*theta1, linestyle='-', lw=3, color='#fc4f30')
    
    ax3.plot(theta0s, theta1s, '-o', markeredgecolor='k', ms=10, zorder=10)
    
    ax4.plot(theta0s, theta1s, losses, '-o', markeredgecolor='k', ms=10)
    
    plt.show()

In [ ]:
alpha = 0.0005

# Launch widget
interact(minimize_plot_2d_model_unscaled,
         epochs=IntSlider(value=1, min=1, max=100000, step=1),
         loss_fn=fixed(mse),
         grad_loss_fn=fixed(grad_mse),
         x=fixed(tips['total_bill'].values),
         y_obs=fixed(tips['tip'].values),
         alpha=fixed(alpha));

# 2D Gradient  Descent (Scaled)

In [24]:
def mse(theta0, theta1, x, y_obs):
    y_hat = theta0 + theta1 * x
    return np.mean((y_hat - y_obs) ** 2)

def grad_mse(theta0, theta1, x, y_obs):
    y_hat = theta0 + theta1 * x
    n = len(x)
    grad_0 = (-2 / n) * sum(y_obs - y_hat)
    grad_1 = (-2 / n) * sum(x * (y_obs - y_hat)) 
    return grad_0, grad_1

In [27]:
def minimize_plot_2d_model_scaled(epochs, loss_fn, grad_loss_fn, x, y_obs, alpha=0.2):
    """
    Uses gradient descent to minimize loss_fn. Returns the minimizing value of
    theta once theta changes less than 0.001 between iterations.
    """
    x = (x - np.mean(x)) / np.std(x)
    y_obs = (y_obs - np.mean(y_obs)) / np.std(y_obs)
    
    # Set starting epoch
    epochs = np.arange(epochs)
    epochs_conv = []
    
    # Set loss array
    losses = []
    losses_conv = []
    
    # Set theta array
    theta0s = []
    theta1s = []
    
    # Setup figure
    fig = plt.figure(figsize=(14, 12))
    fig.subplots_adjust(wspace=0.2, hspace=0.2)
    ax1 = plt.subplot2grid((2, 2), (0, 0))
    ax2 = plt.subplot2grid((2, 2), (0, 1))
    ax3 = plt.subplot2grid((2, 2), (1, 0))
    ax4 = plt.subplot2grid((2, 2), (1, 1), projection='3d')
    
    # Plot total_bill vs tip
    sns.scatterplot(x = x, y = y_obs, ax=ax1)
    ax1.xaxis.set_tick_params(labelsize=14)
    ax1.yaxis.set_tick_params(labelsize=14)
    
    ax1.set_ylabel('Tip', fontsize=18)
    ax1.set_xlabel('Total Bill', fontsize=18)
    ax1.set_xlim([-2, 4])
    ax1.set_ylim([-2, 6])
    
    # Plot 2D contour
    theta0_grid = np.linspace(-10, 15, 200)
    theta1_grid = np.linspace(-10, 10, 200)
    loss_grid = np.zeros([len(theta1_grid), len(theta0_grid)])
    for theta0_id, theta0 in enumerate(theta0_grid):
        for theta1_id, theta1 in enumerate(theta1_grid):
            loss_grid[theta1_id, theta0_id] = loss_fn(theta0, theta1, x, y_obs)
    X, Y = np.meshgrid(theta0_grid, theta1_grid)
    cp = ax3.contour(X, Y, loss_grid, 6, colors='white', linestyles='dashed', linewidths=1)
    ax3.clabel(cp, inline=1, fmt='%1.1f', fontsize=15 )
    ax3.contourf(X, Y, loss_grid, alpha=0.65,cmap=cm.viridis) 
    ax3.set_xlabel('$\\theta_0$', fontsize=18)
    ax3.set_ylabel('$\\theta_1$', fontsize=18)

    surf = ax4.plot_surface(X, Y, loss_grid, cmap='viridis', rstride=1, cstride=1, antialiased=True, alpha=0.65)
    ax4.set_xlabel('$\\theta_0$', fontsize=18)
    ax4.set_ylabel('$\\theta_1$', fontsize=18)
    ax4.set_zlabel('Loss', fontsize=18, rotation=90)
    
    # Set starting theta
    theta0 = 12.5
    theta1 = 7.5
    
    for epoch in epochs:

        # Update losses
        losses.append(loss_fn(theta0, theta1, x, y_obs))
        
        # Update thetas
        theta0s.append(theta0)
        theta1s.append(theta1)
        
        # Compute gradient
        gradient_0, gradient_1 = grad_loss_fn(theta0, 
                                              theta1, 
                                              x, 
                                              y_obs)
        
        # Get new theta
        new_theta0 = theta0 - alpha * gradient_0
        new_theta1 = theta1 - alpha * gradient_1
        
        if abs(new_theta0 - theta0) < 0.001 and abs(new_theta1 - theta1) < 0.001:
            epochs_conv.append(epoch)
            losses_conv.append(loss_fn(theta0, theta1, x, y_obs))
          
        # Update theta
        theta0 = new_theta0
        theta1 = new_theta1
  
    # Check for convergence
    ax2.plot(epochs, losses, '-o', markeredgecolor='k', ms=10)
    ax2.plot(epochs_conv, losses_conv, '-o', markeredgecolor='k', ms=10)
    ax2.set_xlim([-1, 20])
    ax2.set_ylim([-20, 220])
    ax2.set_xlabel('Epoch', fontsize=18)
    ax2.set_ylabel('Loss', fontsize=18)
    ax2.xaxis.set_tick_params(labelsize=14)
    ax2.yaxis.set_tick_params(labelsize=14)
    
    ax1.set_title('$\\theta_0$ = {}\n$\\theta_1$ = {}'.format(np.round(theta0, 2), np.round(theta1, 2)), fontsize=18)
    ax1.plot(x, theta0 + x*theta1, linestyle='-', lw=3, color='#fc4f30')
    if len(losses_conv) > 0:
        ax1.vlines(x, ymin=y_obs, ymax=theta0 + x*theta1, linestyle='dashed', color='r',alpha=0.3, zorder=0)
    
    ax3.plot(theta0s, theta1s, '-o', markeredgecolor='k', ms=10)
    
    ax4.plot(theta0s, theta1s, losses, '-o', markeredgecolor='k', ms=10)
    
    plt.show()

In [ ]:
alpha = 0.2

# Launch widget
interact(minimize_plot_2d_model_scaled,
         epochs=IntSlider(value=1, min=1, max=100, step=1),
         loss_fn=fixed(mse),
         grad_loss_fn=fixed(grad_mse),
         x=fixed(tips['total_bill'].values),
         y_obs=fixed(tips['tip'].values),
         alpha=fixed(alpha));